In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 39.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
from scipy.sparse import csr_matrix
import implicit

# Загрузка данных
train = pd.read_parquet("/train_interactions.parquet")  # Убедитесь, что ссылка корректная
items_meta = pd.read_csv('/cos_sim_output1.csv')
users_meta = pd.read_parquet("/users_meta.parquet.parquet")

In [4]:
del items_meta['embeddings']

In [5]:
items_meta.head()

,item_id,source_id,duration,cosine_similarities
0,0,1869,13,[ 9.99999940e-01 1.68618202e-01 3.62163574e-...
1,1,1869,8,[ 1.68618202e-01 1.00000012e+00 6.20540559e-...
2,2,1869,10,[ 3.62163574e-01 6.20540559e-01 9.99999940e-...
3,3,1869,49,[ 3.08336079e-01 5.46211183e-01 6.53909385e-...
4,4,1869,8,[ 3.81041855e-01 7.44416535e-01 8.41913939e-...


In [6]:
# Фильтрация и добавление веса
train = train.query("(like + dislike) >= 1")
train["weight"] = train["like"] - train["dislike"]
train = train[["user_id", "item_id", "weight"]]

<ipython-input-6-280b1b6bd132>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["weight"] = train["like"] - train["share"]


In [7]:
train.tail()

,user_id,item_id,weight
145667173,153247,209536,1
145667200,21973,291285,1
145667215,129733,183808,1
145667271,81472,274701,1
145667276,153247,220346,1


In [8]:
# Подготовка CSR матрицы
n_items = items_meta["item_id"].max() + 1
n_users = users_meta["user_id"].max() + 1

row = train["user_id"]
col = train["item_id"]
data = train["weight"]

csr_train = csr_matrix((data, (row, col)), shape=(n_users, n_items))

In [9]:
# Обучение модели ALS
model = implicit.als.AlternatingLeastSquares(factors=20,
                                             iterations=20,
                                             regularization=2,
                                             alpha=100,
                                             calculate_training_loss=True)

model.fit(csr_train)

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]

In [10]:
# Прогнозирование на тестовых парах
test_pairs = pd.read_csv('/test_pairs.csv.csv')
def predict(model, user_id, item_id):
    user_factor = model.user_factors[user_id]
    item_factor = model.item_factors[item_id]
    return user_factor.dot(item_factor.T)

# Используем функцию для получения предсказаний
predictions = []
for index, row in test_pairs.iterrows():
    user_id = row['user_id']
    item_id = row['item_id']

    prediction = predict(model, user_id, item_id)
    predictions.append(prediction)

# Добавляем предсказания к DataFrame и сохраняем результат
test_pairs['predict'] = predictions
test_pairs.to_csv('/sample_submission(cos_sim1).csv', index=False)

In [11]:
test_pairs.head()

,user_id,item_id,predict
0,1,7363,0.861153
1,1,73770,0.667081
2,1,75700,0.264079
3,1,81204,0.956124
4,1,110249,0.549416


In [ ]:
test_pairs.head()

,user_id,item_id,predict
0,1,7363,0.432516
1,1,73770,0.218033
2,1,75700,0.359592
3,1,81204,0.520789
4,1,110249,0.159195
